In [1]:
import pandas as pd 
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
final_df = pd.read_csv("/Users/george/Desktop/LW-DDI Project/final_df.csv")

In [3]:
final_df

,Unnamed: 0,Drug1,Drug2,DD_ID,0,1,2,3,4,5,...,SRW09_diff,SRW10_diff,TSRW10_diff,MW_diff,AMW_diff,WPath_diff,WPol_diff,Zagreb1_diff,Zagreb2_diff,mZagreb2_diff
0,0,100,104,100104,0,1,1,1,1,1,...,0.0000,0.0063,3.2221,54.9921,0.5812,1253,2,10,7,0.6528
1,1,100,105,100105,0,1,1,0,1,0,...,-6.8233,0.5150,-2.6891,143.9881,0.9824,2124,16,52,59,1.5972
2,2,100,111,100111,1,1,1,1,1,0,...,-6.5294,-0.1611,-5.1299,94.1436,-2.0755,2315,3,30,23,1.9306
3,3,100,117,100117,0,1,1,1,1,0,...,0.0000,0.2369,10.4883,135.9844,1.3654,2262,13,44,50,2.1111
4,4,100,121,100121,0,1,1,1,1,0,...,0.0000,0.3873,9.7546,127.9793,1.3315,2152,12,44,50,1.6528
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63468,63468,744,452,744452,0,1,0,0,0,0,...,0.3029,0.3688,7.7574,73.1321,-1.4971,889,15,30,39,1.4722
63469,63469,744,460,744460,0,1,1,0,1,0,...,0.7722,-0.0189,0.0662,-4.9803,-0.5157,-350,-1,-4,-4,-0.2500
63470,63470,744,571,744571,0,0,0,0,1,0,...,7.0656,-0.1618,11.6557,-42.9112,-1.1375,-1016,-3,-18,-19,-0.0556
63471,63471,744,697,744697,0,1,1,0,1,0,...,7.0656,0.1107,23.3410,122.1419,-1.2809,1680,10,36,40,1.9444


In [4]:
final_df.describe()

,Unnamed: 0,Drug1,Drug2,DD_ID,0,1,2,3,4,5,...,SRW09_diff,SRW10_diff,TSRW10_diff,MW_diff,AMW_diff,WPath_diff,WPol_diff,Zagreb1_diff,Zagreb2_diff,mZagreb2_diff
count,63473.000000,63473.000000,63473.000000,63473.000000,63473.000000,63473.000000,63473.000000,63473.000000,63473.000000,63473.000000,...,63473.000000,63473.000000,63473.000000,63473.000000,63473.000000,6.347300e+04,63473.000000,63473.00000,63473.000000,63473.000000
mean,31736.000000,406.112709,420.029792,406532.739133,0.284625,0.927512,0.666630,0.494620,0.866841,0.196383,...,-0.349763,-0.020920,-2.980403,-25.297638,0.214177,-1.479215e+09,-3.123879,-9.73551,-11.583035,-0.381327
std,18323.221155,181.461067,185.561598,181460.083411,0.451239,0.259295,0.471421,0.499975,0.339749,0.397264,...,4.893456,1.883625,27.475811,287.122227,4.922274,1.671228e+10,37.798473,103.03090,125.255364,4.133774
min,0.000000,100.000000,100.000000,100104.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,-8.963500,-11.688200,-153.663800,-1583.936800,-50.198000,-3.782000e+11,-229.000000,-626.00000,-768.000000,-24.486100
25%,15868.000000,249.000000,265.000000,249552.000000,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,...,-5.459600,-0.653300,-17.608400,-135.068400,-1.126500,-1.702000e+03,-20.000000,-56.00000,-71.000000,-2.111100
50%,31736.000000,405.000000,408.000000,405632.000000,0.000000,1.000000,1.000000,0.000000,1.000000,0.000000,...,0.000000,-0.071700,-2.730800,-20.044400,0.082700,-2.820000e+02,-3.000000,-8.00000,-10.000000,-0.319400
75%,47604.000000,566.000000,573.000000,566167.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,...,1.269100,0.472800,11.739600,96.234800,1.384200,8.640000e+02,15.000000,38.00000,49.000000,1.500000
max,63472.000000,744.000000,744.000000,744724.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,8.963500,11.688200,152.663800,1475.780800,49.793200,1.440000e+11,201.000000,550.00000,662.000000,21.625000


In [5]:
final_df['DD_ID']

0        100104
1        100105
2        100111
3        100117
4        100121
          ...  
63468    744452
63469    744460
63470    744571
63471    744697
63472    744724
Name: DD_ID, Length: 63473, dtype: int64